In [1]:
import os
import re
import numpy as np
import textwrap
import torch
from accelerate import init_empty_weights, Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
from custom_modeling_opt import CustomOPTForCausalLM
from matplotlib import pyplot as plt

/home/menuab/miniconda3/envs/chemlactica/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed_value=42
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
import rdkit.Chem as Chem
import sys
from rdkit.Chem import RDConfig, MACCSkeys, QED
from rdkit.Chem.rdMolDescriptors import CalcTPSA, CalcCrippenDescriptors
from rdkit.Chem import Descriptors
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer

In [4]:
device = "cuda:1"

In [5]:
tokenizer = AutoTokenizer.from_pretrained("/home/menuab/code/ChemLacticaTestSuite/src/tokenizer/ChemLacticaTokenizer_50066/")
print('tokenizer size: ', len(tokenizer))

tokenizer size:  50066


In [6]:
checkpoint_path = "/home/menuab/code/checkpoints/f2c6ebb289994595a478f513/125m_126k_f2c6/"
checkpoint_path = "/home/menuab/code/checkpoints/f3fbd012918247a388efa732/125m_126k_f3fb/"
checkpoint_path

'/home/menuab/code/checkpoints/f3fbd012918247a388efa732/125m_126k_f3fb/'

In [8]:
model = CustomOPTForCausalLM.from_pretrained(
            checkpoint_path,
            use_flash_attn=True,
            torch_dtype=torch.bfloat16
            )
model.eval()
model.to(device)
print(f'model loaded with embedding size of : {model.model.decoder.embed_tokens.num_embeddings}')
assert(model.model.decoder.embed_tokens.num_embeddings == len(tokenizer))

model loaded with embedding size of : 50066


In [9]:
left = "[START_SMILES]"
right = "[END_SMILES]"

In [11]:
sac_targets1, sac_targets2, sac_scores, sac_scores2, sac_invalids = [], [], [], [], []
for target1 in np.arange(1.1, 10.10, .2):
    for target2 in np.arange(1.1, 10.10, .2):
        prompt = f"</s>[SAS]{target1:.2f}[/SAS][CLOGP]{target2:.2f}[/CLOGP]"
        inputs = tokenizer(prompt, return_tensors="pt").input_ids
        inputs = inputs.to(device)
        context_length = inputs.shape[1]
        out = model.generate(inputs, max_new_tokens=300, do_sample=False,
                            #   top_p = 1.0, 
                            #   num_return_sequences=6,
                            #  num_beams=6, length_penalty=-5, renormalize_logits=True,
                            #   repetition_penalty=1.5, diversity_penalty=1., num_beam_groups=3, 
                            eos_token_id=20, return_dict_in_generate=True, output_scores=True)

        for seq in out.sequences:
            decoded = tokenizer.decode(seq)
            print('--------')
            try:
                left = "[START_SMILES]"
                right = "[END_SMILES]"
                captured_text = decoded[decoded.find("[START_SMILES]")+len("[START_SMILES]"):decoded.find("[END_SMILES]")]
                print(decoded)
                # captured_text = re.match("^.*?(?=\\[END_SMILES])", tokenizer2.decode(out.sequences[0, context_length:])).group()
                mol_source = Chem.MolFromSmiles(captured_text)
                # score = Chem.Descriptors.HeavyAtomCount(mol_source)
                sas_score = sascorer.calculateScore(mol_source) 
                # qed_score = round(QED.qed(mol_source), 3)
                clogp_score = round(Descriptors.MolLogP(mol_source), 3)
                # weight_score = round(Descriptors.ExactMolWt(mol_source), 3)
                print(f'{target1=:.2f}', f'{target2=:.2f}', f'{sas_score=:.2f}', f'{clogp_score=:.2f}', captured_text)
                sac_targets1.append(target1)
                sac_targets2.append(target2)
                sac_scores.append(sas_score)
                sac_scores2.append(clogp_score)
                break
            except:
                sac_invalids.append((target1, target2))
        # print(f'{score:.2f}', decoded)
        # break

--------
</s>[SAS]1.10[/SAS][CLOGP]1.10[/CLOGP][FRACTIONCSP3]0.22[/FRACTIONCSP3][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][WEIGHT]205.09[/WEIGHT][NUMHACCEPTORS]3[/NUMHACCEPTORS][RINGCOUNT]1[/RINGCOUNT][NUMHDONORS]1[/NUMHDONORS][NUMROTATABLEBONDS]4[/NUMROTATABLEBONDS][NUMAROMATICRINGS]1[/NUMAROMATICRINGS][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][QED]0.75[/QED][NUMHETEROATOMS]4[/NUMHETEROATOMS][IUPAC]2-(4-methoxyphenyl)-N-(2-oxoethyl)acetamide[/IUPAC][NOCOUNT]4[/NOCOUNT][HEAVYATOMCOUNT]15[/HEAVYATOMCOUNT][SIMILAR]CC(=O)NCC(=O)NCC1=CC=C(C=C1)OC 0.82[/SIMILAR][SIMILAR]CC(C)(C)C(=O)NCC(=O)NCC1=CC=C(C=C1)OC 0.74[/SIMILAR][SIMILAR]CC(C)(C)C(=O)NCC(=O)NCC1=CC=C(C=C1)OC 0.74[/SIMILAR][SIMILAR]CC(C)(C)C(=O)NCC(=O)NCC1=CC=C(C=C1)OC 0.74[/SIMILAR][SIMILAR]CC(C)(C)C(=O)NCC(=O)NCC1=CC=C(C=C1)OC 0.74[/SIMILAR][SIMILAR]CC(C)(C)C(=O)NCC(=O)NCC1=CC=C(C=C1)OC 0.


[17:35:56] SMILES Parse Error: syntax error while parsing: [/SAS][CLOGP]1.10[/CLOGP][FRACTIONCSP3]0.22[/FRACTIONCSP3][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][WEIGHT]205.09[/WEIGHT][NUMHACCEPTORS]3[/NUMHACCEPTORS][RINGCOUNT]1[/RINGCOUNT][NUMHDONORS]1[/NUMHDONORS][NUMROTATABLEBONDS]4[/NUMROTATABLEBONDS][NUMAROMATICRINGS]1[/NUMAROMATICRINGS][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][QED]0.75[/QED][NUMHETEROATOMS]4[/NUMHETEROATOMS][IUPAC]2-(4-methoxyphenyl)-N-(2-oxoethyl)acetamide[/IUPAC][NOCOUNT]4[/NOCOUNT][HEAVYATOMCOUNT]15[/HEAVYATOMCOUNT][SIMILAR]CC(=O)NCC(=O)NCC1=CC=C(C=C1)OC
[17:35:56] SMILES Parse Error: Failed parsing SMILES '[/SAS][CLOGP]1.10[/CLOGP][FRACTIONCSP3]0.22[/FRACTIONCSP3][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][WEIGHT]205.09[/WEIGHT][NUMHACCEPTORS]3[/NUMHACCEPTORS][RINGCOUNT]1[/R

--------
</s>[SAS]1.10[/SAS][CLOGP]1.30[/CLOGP][FRACTIONCSP3]0.25[/FRACTIONCSP3][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][WEIGHT]221.11[/WEIGHT][NUMHACCEPTORS]3[/NUMHACCEPTORS][NUMROTATABLEBONDS]4[/NUMROTATABLEBONDS][RINGCOUNT]1[/RINGCOUNT][NUMHDONORS]2[/NUMHDONORS][NUMAROMATICRINGS]1[/NUMAROMATICRINGS][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][QED]0.75[/QED][NUMHETEROATOMS]4[/NUMHETEROATOMS][IUPAC]N-[2-(aminomethyl)phenyl]-2-methoxyacetamide[/IUPAC][NOCOUNT]4[/NOCOUNT][HEAVYATOMCOUNT]16[/HEAVYATOMCOUNT][NUMSATURATEDHETEROCYCLES]0[/NUMSATURATEDHETEROCYCLES][START_SMILES]COCC(=O)NC1=CC=CC=C1CN[END_SMILES]
target1=1.10 target2=1.30 sas_score=1.73 clogp_score=0.73 COCC(=O)NC1=CC=CC=C1CN
--------
</s>[SAS]1.10[/SAS][CLOGP]1.50[/CLOGP][FRACTIONCSP3]0.25[/FRACTIONCSP3][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOC

[17:35:59] SMILES Parse Error: syntax error while parsing: [/SAS][CLOGP]2.10[/CLOGP][FRACTIONCSP3]0.13[/FRACTIONCSP3][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][RINGCOUNT]2[/RINGCOUNT][WEIGHT]255.11[/WEIGHT][NUMHACCEPTORS]3[/NUMHACCEPTORS][NUMHDONORS]1[/NUMHDONORS][NUMROTATABLEBONDS]4[/NUMROTATABLEBONDS][NUMAROMATICRINGS]2[/NUMAROMATICRINGS][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][QED]0.85[/QED][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][NUMHETEROATOMS]4[/NUMHETEROATOMS][IUPAC]N-[(4-methoxyphenyl)methyl]-4-methylbenzenesulfonamide[/IUPAC][SIMILAR]CC1=CC=C(C=C1)S(=O)(=O)NCC2=CC=C(C=C2)OC
[17:35:59] SMILES Parse Error: Failed parsing SMILES '[/SAS][CLOGP]2.10[/CLOGP][FRACTIONCSP3]0.13[/FRACTIONCSP3][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][RINGCOUNT]2[/RINGCOUNT][WEIGHT]255.11[/WEIGHT][NUMHACCEPTORS]3[/NUMHACCEPTORS][NUMHDONORS]1[/NUMHDONORS

--------
</s>[SAS]1.10[/SAS][CLOGP]2.30[/CLOGP][FRACTIONCSP3]0.13[/FRACTIONCSP3][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][RINGCOUNT]2[/RINGCOUNT][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][WEIGHT]255.11[/WEIGHT][NUMHACCEPTORS]3[/NUMHACCEPTORS][NUMHDONORS]1[/NUMHDONORS][NUMROTATABLEBONDS]4[/NUMROTATABLEBONDS][NUMAROMATICRINGS]2[/NUMAROMATICRINGS][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][QED]0.85[/QED][NUMHETEROATOMS]3[/NUMHETEROATOMS][IUPAC]N-[(4-methoxyphenyl)methyl]-2-phenylacetamide[/IUPAC][SIMILAR]CC(C)(C)C(=O)NCC1=CC=C(C=C1)OC 0.66[/SIMILAR][SIMILAR]CC(C)(C)C(=O)NCC1=CC=C(C=C1)OC 0.66[/SIMILAR][SIMILAR]CC(C)(C)C(=O)NCC1=CC=C(C=C1)OC 0.66[/SIMILAR][SIMILAR]CC(C)(C)C(=O)NCC1=CC=C(C=C1)OC 0.66[/SIMILAR][SIMILAR]CC(C)(C)C(=O)NCC1=CC=C(C=C1)OC 0.66[/SIMILAR][SIMILAR]CC(C)(C)C(=O)NCC1=CC=C(C=C1)OC 0.66[/SIMILAR][SIMILAR]CC(C)(C)C(=O)NCC1=CC=C(C=C1)OC 0.66[/SIMILAR][SIMILAR]CC


...

[17:36:22] SMILES Parse Error: syntax error while parsing: [/SAS][CLOGP]7.70[/CLOGP][FRACTIONCSP3]0.00[/FRACTIONCSP3][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMSATURATEDRINGS]0[/NUMSATURATEDRINGS][NUMHACCEPTORS]0[/NUMHACCEPTORS][IUPAC]1-phenyl-4-(4-phenylphenyl)benzene[/IUPAC][QED]0.33[/QED][SIMILAR]C1=CC=C(C=C1)C2=CC=C(C=C2)C3=CC=C(C=C3)C4=CC=C(C=C4)C5=CC=C(C=C5)C6=CC=C(C=C6)C7=CC=C(C=C7)C8=CC=C(C=C8)C9=CC=C(C=C9)C1=CC=C(C=C1)C1=CC=CC=C1
[17:36:22] SMILES Parse Error: Failed parsing SMILES '[/SAS][CLOGP]7.70[/CLOGP][FRACTIONCSP3]0.00[/FRACTIONCSP3][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMSATURATEDRINGS]0[/NUMSATURATEDRINGS][NUMHACCEPTORS]0[/NUMHACCEPTORS][IUPAC]1-phenyl-4-(4-phenylphenyl)benzene[/IUPAC][QED]0.33[/QED][SIMILAR]C1=CC=C(C=C1)C2=CC=C(C=C2)C3=CC=C(C=C3)C4=CC=C(C=C4)C5=CC=C(C=C5)C6=CC=C(C=C6)C7=CC=C(C=C7)C8=CC=C(C=C8)C9=CC=C(C=C9)C1=CC=C(C=

--------
</s>[SAS]1.10[/SAS][CLOGP]7.90[/CLOGP][FRACTIONCSP3]0.00[/FRACTIONCSP3][NUMAROMATICRINGS]4[/NUMAROMATICRINGS][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][RINGCOUNT]4[/RINGCOUNT][NUMSATURATEDRINGS]0[/NUMSATURATEDRINGS][NHOHCOUNT]0[/NHOHCOUNT][WEIGHT]388.13[/WEIGHT][IUPAC]2-[4-(4-fluorophenyl)phenyl]-4-phenyl-6-(trifluoromethyl)pyridine[/IUPAC][QED]0.31[/QED][NUMHACCEPTORS]1[/NUMHACCEPTORS][NUMHDONORS]0[/NUMHDONORS][NUMROTATABLEBONDS]3[/NUMROTATABLEBONDS][SIMILAR]C1=CC=C(C=C1)C2=CC(=NC(=C2)C3=CC=CC=C3)C4=CC=CC=C4 0.67[/SIMILAR][SIMILAR]C1=CC=C(C=C1)C2=CC(=CC(=N2)C3=CC=CC=C3)C4=CC=C(C=C4)C5=CC=C(C=C5)C6=CC(=NC(=C6)C7=CC=CC=C7)C8=CC=CC=C8 0.6[/SIMILAR][SIMILAR]C1=CC=C(C=C1)C2=CC(=CC(=N2)C3=CC=CC=C3)C4=CC=C(C=C4)C5=CC=C(C=C5)C6=CC(=NC(=C6)C7=CC=CC=C7)C8


[17:36:23] SMILES Parse Error: syntax error while parsing: [/SAS][CLOGP]7.90[/CLOGP][FRACTIONCSP3]0.00[/FRACTIONCSP3][NUMAROMATICRINGS]4[/NUMAROMATICRINGS][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][RINGCOUNT]4[/RINGCOUNT][NUMSATURATEDRINGS]0[/NUMSATURATEDRINGS][NHOHCOUNT]0[/NHOHCOUNT][WEIGHT]388.13[/WEIGHT][IUPAC]2-[4-(4-fluorophenyl)phenyl]-4-phenyl-6-(trifluoromethyl)pyridine[/IUPAC][QED]0.31[/QED][NUMHACCEPTORS]1[/NUMHACCEPTORS][NUMHDONORS]0[/NUMHDONORS][NUMROTATABLEBONDS]3[/NUMROTATABLEBONDS][SIMILAR]C1=CC=C(C=C1)C2=CC(=NC(=C2)C3=CC=CC=C3)C4=CC=CC=C4
[17:36:23] SMILES Parse Error: Failed parsing SMILES '[/SAS][CLOGP]7.90[/CLOGP][FRACTIONCSP3]0.00[/FRACTIONCSP3][NUMAROMATICRINGS]4[/NUMAROMATICRINGS][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][RINGCOUNT]4[/RINGCOUNT][NUMSATURATEDRINGS]0[/NUMSATURATEDRINGS][NHOHCOUNT]0[/NHOHCOUNT][WEIGHT]388.13[/WEIGHT][IUPAC]2

--------
</s>[SAS]1.10[/SAS][CLOGP]8.10[/CLOGP][FRACTIONCSP3]0.00[/FRACTIONCSP3][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMSATURATEDRINGS]0[/NUMSATURATEDRINGS][IUPAC]N-[4-(4-naphthalen-1-ylphenyl)phenyl]-N-phenylaniline[/IUPAC][NUMALIPHATICRINGS]0[/NUMALIPHATICRINGS][NUMHACCEPTORS]1[/NUMHACCEPTORS][NUMAROMATICRINGS]6[/NUMAROMATICRINGS][QED]0.25[/QED][NUMHDONORS]0[/NUMHDONORS][NUMROTATABLEBONDS]5[/NUMROTATABLEBONDS][RINGCOUNT]6[/RINGCOUNT][WEIGHT]461.20[/WEIGHT][NUMHETEROATOMS]1[/NUMHETEROATOMS][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][SIMILAR]C1=CC=C(C=C1)C2=CC=C(C=C2)N(C3=CC=C(C=C3)C4=CC=CC=C4)C5=CC=C(C=C5)C6=CC=C(C=C6)N(C7=CC=C(C=C7)C8=CC=CC=C8)C9=CC=C(C=C9)C1=CC=CC2=CC=CC=C21 0.94[/SIMILAR][SIMILAR]C1=CC=C(C=C1)C2=CC=C(C=C2)N(C3=CC=C(C=C3)C4=CC=CC=C4)C5=CC=C(C=C5)C6=CC=C(C=C6)N(C7=CC=C(C=C7)C


[17:36:24] SMILES Parse Error: syntax error while parsing: [/SAS][CLOGP]8.10[/CLOGP][FRACTIONCSP3]0.00[/FRACTIONCSP3][NUMALIPHATICCARBOCYCLES]0[/NUMALIPHATICCARBOCYCLES][NUMALIPHATICHETEROCYCLES]0[/NUMALIPHATICHETEROCYCLES][NUMSATURATEDRINGS]0[/NUMSATURATEDRINGS][IUPAC]N-[4-(4-naphthalen-1-ylphenyl)phenyl]-N-phenylaniline[/IUPAC][NUMALIPHATICRINGS]0[/NUMALIPHATICRINGS][NUMHACCEPTORS]1[/NUMHACCEPTORS][NUMAROMATICRINGS]6[/NUMAROMATICRINGS][QED]0.25[/QED][NUMHDONORS]0[/NUMHDONORS][NUMROTATABLEBONDS]5[/NUMROTATABLEBONDS][RINGCOUNT]6[/RINGCOUNT][WEIGHT]461.20[/WEIGHT][NUMHETEROATOMS]1[/NUMHETEROATOMS][NUMSATURATEDCARBOCYCLES]0[/NUMSATURATEDCARBOCYCLES][NUMAROMATICHETEROCYCLES]0[/NUMAROMATICHETEROCYCLES][SIMILAR]C1=CC=C(C=C1)C2=CC=C(C=C2)N(C3=CC=C(C=C3)C4=CC=CC=C4)C5=CC=C(C=C5)C6=CC=C(C=C6)N(C7=CC=C(C=C7)C8=CC=CC=C8)C9=CC=C(C=C9)C1=CC=CC2=CC=CC=C21
[17:36:24] SMILES Parse Error: Failed parsing SMILES '[/SAS][CLOGP]8.10[/CLOGP][FRACTIONCSP3]0.00[/FRACTIONCSP3][NUMALIPHATICCARBOCYCLES]0[/NUMAL

In [ ]:
cas_targets1, cas_targets2, cas_scores, cas_scores2, cas_invalids = [], [], [], [], []
for target1 in np.arange(1.1, 10.10, .2):
    for target2 in np.arange(1.1, 10.10, .2):
        prompt = f"</s>[SAS]{target1:.2f}[/SAS][CLOGP]{target2:.2f}[/CLOGP]"
        inputs = tokenizer(prompt, return_tensors="pt").input_ids
        inputs = inputs.to(device)
        context_length = inputs.shape[1]
        out = model.generate(inputs, max_new_tokens=300, do_sample=False,
                            #   top_p = 1.0, 
                            #   num_return_sequences=6,
                            #  num_beams=6, length_penalty=-5, renormalize_logits=True,
                            #   repetition_penalty=1.5, diversity_penalty=1., num_beam_groups=3, 
                            eos_token_id=20, return_dict_in_generate=True, output_scores=True)

        for seq in out.sequences:
            decoded = tokenizer.decode(seq)
            print('--------')
            try:
                left = "[START_SMILES]"
                right = "[END_SMILES]"
                captured_text = decoded[decoded.find("[START_SMILES]")+len("[START_SMILES]"):decoded.find("[END_SMILES]")]
                print(decoded)
                # captured_text = re.match("^.*?(?=\\[END_SMILES])", tokenizer2.decode(out.sequences[0, context_length:])).group()
                mol_source = Chem.MolFromSmiles(captured_text)
                # score = Chem.Descriptors.HeavyAtomCount(mol_source)
                sas_score = sascorer.calculateScore(mol_source) 
                # qed_score = round(QED.qed(mol_source), 3)
                clogp_score = round(Descriptors.MolLogP(mol_source), 3)
                # weight_score = round(Descriptors.ExactMolWt(mol_source), 3)
                print(f'{target1=:.2f}', f'{target2=:.2f}', f'{sas_score=:.2f}', f'{clogp_score=:.2f}', captured_text)
                cas_targets1.append(target1)
                cas_targets2.append(target2)
                cas_scores.append(sas_score)
                cas_scores2.append(clogp_score)
                break
            except:
                cas_invalids.append((target1, target2))
        # print(f'{score:.2f}', decoded)
        # break